# **Funciones de Negocio**

In [2]:
import pandas as pd

In [1]:
# Revisar la cantidad de tipos de datos y valores nulos por cada columna de un dataframe

def count_column_types(df):
    '''
    Obtener los tipos de datos unicos y sus respectivos recuentos para cada columna.
    Funcion que recibe como parámetro un dataframe
    '''
    
    result = df.apply(lambda col: col.apply(type) .value_counts())          # Aplicar la función a todas las columnas del DataFrame
    result = result.T.reset_index()                                         # Transponer el resultado y restablecer el índice
    result. columns = ["Column"] + result.columns[1:].tolist()              # Renombrar las columnas

    serie_nulos = df.isnull().sum()                                         # Hallamos los valores nulos y se almacenan en una serie
    serie_nulos = serie_nulos.reset_index()                                 # Convierte el índice en una columna
    serie_nulos.columns = ['Column', 'Nulos']                               # Asigna un nombre a la columna del índice si es necesario

    df_1 = result
    df_2 = serie_nulos
    columna_union = "Column"
    df_merge = df_1.merge(df_2, on=columna_union, how='left')               # Realizar un left join entre df_userdata y df_games

    df_merge["Nulos %"] = round(df_merge["Nulos"] / df.shape[0] * 100, 2)   # Porcentaje total de registros nulos

    print("\nfilas completamente nulas: ", df.isna().all(axis=1).sum())     # Filas que se encuentran totalmente en nulo

    return df_merge

## **1- def userdata**

- Entrada: (user_id : str)
- Objetivo: Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [ ]:
def userdata (user_id:str):
    '''
    Función que recibe de entrada el ID de un usuario y devuelve:
    - Cantidad total de dinero que ha gastado en items
    - Cantidad total de items que posee
    - Porcentaje de items que recomienda respecto al total de items que posee.
    '''
    # Lectura de CSV en un df
    ruta = "eda_userdata.csv"
    df_userdata = pd.read_csv(ruta)

    user_id = str(user_id)
    user_id = user_id.strip()

    df_filtrado = df_userdata[df_userdata["user_id"] == user_id]
    mi_dict = df_filtrado.to_dict(orient='records')

    return mi_dict[0]

In [ ]:
userdata("76561197970982479")

{'user_id': '76561197970982479',
 'items_count': 277,
 'money_spent': 3424.31,
 'total_recommend': 3.0,
 'recommend_percent': 1.08}

## **2- def countreviews**

- Entrada: (YYYY-MM-DD : str , YYYY-MM-DD : str)
- Objetivo: Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [81]:
def countreviews (desde:str, hasta:str):
    '''
    Función que recibe dos fechas de entrada en formato YYYY-MM-DD y devuelve:
    - Cantidad de usuarios que realizaron reviews entre las fechas dadas
    - Porcentaje de items que se recomiendan de acuerdo a todas las reviews que se realizaron entre esas fechas.
    '''
    # Lectura de CSV en un df
    ruta = "eda_countreviews.csv"
    df_countreviews = pd.read_csv(ruta)

    desde = str(desde)
    desde = desde.strip().replace("-","")
    desde = int(desde)
    
    hasta = str(hasta)
    hasta = hasta.strip().replace("-","")
    hasta = int(hasta)

    df_filtrado = df_countreviews[(df_countreviews["posted_date_num"] >= desde) &
                                    (df_countreviews["posted_date_num"] <= hasta)]

    user_count = df_filtrado["user_id"].nunique()
    recommend_count = df_filtrado["recommend"].count()
    recommend_sum = df_filtrado["recommend"].sum()
    recommend_percent = round(recommend_sum / recommend_count *100, 2)

    mi_dict = {"user_count": user_count, 
                "recommend_percent": recommend_percent
                }

    return mi_dict

In [82]:
countreviews("2011-04-05", "2011-06-08")

{'user_count': 34, 'recommend_percent': 97.56}

## **3- def genre**

- Entrada: (genero : str)
- Objetivo: Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [199]:
def genre(genre:str):
    '''
    Función que recibe de entrada un género de juego y devuelve:
    - Lugar que ocupa dicho género en un ranking de todos los géneros de acuerdo al tiempo total de juego para cada género.
    
    Nota: El género con mayor cantidad de tiempo jugado ocupará la posición #1 del ranking
    '''
    # Lectura de CSV en un df
    ruta = "eda_genre.csv"
    df_genre = pd.read_csv(ruta)

    look_genre = genre.lower().replace(" ", "")

    df_filtrado = df_genre[["Genre", "Ranking"]][df_genre["look_genre"] == look_genre]
    mi_dict = df_filtrado.to_dict(orient='records')

    return mi_dict[0]

In [147]:
genre("indiE")

{'Genre': 'Indie', 'Ranking': 2}

## **4- def userforgenre**

- Entrada: (género : str)
- Objetivo: Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

In [301]:
def userforgenre(genre:str):
    '''
    Función que recibe de entrada un género de juego y devuelve:
    - Lista de 5 usuarios con mayor cantidad de horas de juego para dicho género.
    - URL de cada usuario
    
    Nota: El usuario con mayor cantidad de tiempo jugado ocupará la posición #1 del ranking
    '''
    # Lectura de CSV en un df
    ruta = "eda_userforgenre.csv"
    df_userforgenre = pd.read_csv(ruta)

    look_genre = genre.lower().replace(" ", "")

    df_filtrado = df_userforgenre[["ranking", "user_id", "user_url"]][df_userforgenre["look_genre"] == look_genre]
    mi_list = df_filtrado.to_dict(orient='records')

    claves = []
    for indice in range (0,5):
        valor = "Ranking_" + str(indice + 1)
        claves.append(valor)

    mi_dict = {}

    for indice, clave in enumerate(claves):
        user_id = df_filtrado['user_id'].iloc[indice]
        url_user = df_filtrado["user_url"].iloc[indice]
        registro = "user_id: " + user_id + " // url_user: " + url_user
        mi_dict[clave] = registro

    return mi_dict

In [302]:
userforgenre("action")

{'Ranking_1': 'user_id: Sp3ctre // url_user: http://steamcommunity.com/id/Sp3ctre',
 'Ranking_2': 'user_id: shinomegami // url_user: http://steamcommunity.com/id/shinomegami',
 'Ranking_3': 'user_id: REBAS_AS_F-T // url_user: http://steamcommunity.com/id/REBAS_AS_F-T',
 'Ranking_4': 'user_id: Terminally-Chill // url_user: http://steamcommunity.com/id/Terminally-Chill',
 'Ranking_5': 'user_id: DownSyndromeKid // url_user: http://steamcommunity.com/id/DownSyndromeKid'}

## **5- def developer**

- Entrada: (desarrollador : str)
- Objetivo: Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

In [271]:
def developer(developer:str):
    '''
    Función que recibe de entrada el nombre de un desarrollador y devuelve:
    - Lista de años en los que el desarrollador hizo lanzamiento de items
    - Cantidad de items por año
    - Porcentaje de items free respecto al total de items lanzados ese el año
    '''
    # Lectura de CSV en un df
    ruta = "eda_developer.csv"
    df_developer = pd.read_csv(ruta)

    look_developer = developer.lower().replace(" ", "")

    df_filtrado = df_developer[["release_year", "items_count", "free_percent"]][df_developer["look_developer"] == look_developer]
    df_filtrado = df_filtrado.reset_index()

    lista_claves = df_filtrado['release_year'].tolist()
    mi_dict = {}

    for indice, clave in enumerate(lista_claves):
        items_count = df_filtrado['items_count'].iloc[indice]
        free_percent = df_filtrado["free_percent"].iloc[indice]
        registro = "items_count: " + str(items_count) + " // free_percent: " + str(free_percent)
        mi_dict[clave] = registro

    return mi_dict

In [272]:
developer("07th Expansion")

{2015.0: 'items_count: 2 // free_percent: 0.0',
 2016.0: 'items_count: 3 // free_percent: 0.0',
 2017.0: 'items_count: 3 // free_percent: 0.0'}

## **6- def sentiment_analysis**

- Entrada: (año : int)
- Objetivo: Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [308]:
def sentiment_analysis(year:str):
    '''
    Función que recibe de entrada un año y devuelve:
    - Listado de categorias basadas en el analisis de sentimiento de las reseñas realizadas en ese año de los items
    - Cantidad total de reseñas para cada categoría de analisis de sentimiento
    '''
    # Lectura de CSV en un df
    ruta = "eda_sentiment_analysis.csv"
    df_sentiment_analysis = pd.read_csv(ruta)

    year = float(year)

    df_filtrado = df_sentiment_analysis[["sentiment_label", "count"]][df_sentiment_analysis["release_year"] == year]
    df_filtrado = df_filtrado.groupby(["sentiment_label"])["count"].sum().reset_index()
    mi_dict = df_filtrado.to_dict(orient='records')

    lista_claves = df_filtrado['sentiment_label'].tolist()
    mi_dict = {}

    for indice, clave in enumerate(lista_claves):
        mi_dict[clave] = str(df_filtrado["count"].iloc[indice])

    return mi_dict

In [309]:
sentiment_analysis("1990")

{'negative': '1', 'positive': '4'}